In [1]:
import pandas as pd
import aiohttp
import aiomoex

/tmp/ipykernel_11764/2646848745.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Скачаем исторические данные о торгах на Московской бирже за исследуемый период (2018 г.)

In [2]:
async def get_moex_data(start_date, end_date):
    history_data = pd.DataFrame()
    dates = pd.date_range(start_date, end_date, freq='D')
    
    async with aiohttp.ClientSession() as session:
        
        for date in dates:
            date = date.strftime('%Y-%m-%d')
            
            prefix="https://iss.moex.com"
            url = f'/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json?date={date}'
            
            # filtering columns of downloaded data
            # arguments = {"history.columns": ("BOARDID," "TRADEDATE," "SHORTNAME," "SECID," "NUMTRADES," "VALUE,"
            #                                     "OPEN," "LOW," "HIGH," "LEGALCLOSEPRICE," "WAPRICE," "CLOSE," "VOLUME")}
            
            iss = aiomoex.ISSClient(session, 
                                    prefix + url,
                                    # arguments
                                    )
            data = await iss.get_all()
            data_df = pd.DataFrame(data['history'])
            history_data = pd.concat([history_data, data_df], axis=0, ignore_index=True)
        
        return history_data

history_data = await get_moex_data(start_date='2018-01-01', end_date='2018-12-31')
history_data

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,TQBR,2018-01-03,БанкАстаны,ABBN,6,130021.2,228.000,228.00,229.28,228.20,...,NaN,228.38,228.20,0.0,1128208.8,0.0,None,3,SUR,-0.68
1,TQBR,2018-01-03,АбрауДюрсо,ABRD,5,70500.0,140.000,139.00,143.00,139.00,...,151.50,151.50,139.00,591650.0,515000.0,591650.0,None,3,SUR,-4.79
2,TQBR,2018-01-03,АСКО ао,ACKO,0,0.0,NaN,NaN,NaN,6.51,...,NaN,6.52,6.51,0.0,586920.0,0.0,None,3,SUR,NaN
3,TQBR,2018-01-03,Система ао,AFKS,5071,256119052.0,12.485,12.26,12.55,12.35,...,12.38,12.38,12.35,256119052.0,256119052.0,256119052.0,None,3,SUR,2.15
4,TQBR,2018-01-03,Аэрофлот,AFLT,3163,296982430.0,139.100,138.60,140.95,140.40,...,139.65,139.65,140.40,296982430.0,296982430.0,296982430.0,None,3,SUR,1.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70820,TQBR,2018-12-29,Yandex clA,YNDX,3314,91724595.5,1920.500,1920.00,1934.00,1930.00,...,1928.00,1928.00,1930.00,91724595.5,91724595.5,91724595.5,None,3,SUR,0.73
70821,TQBR,2018-12-29,ТНСэнЯр,YRSB,0,0.0,NaN,NaN,NaN,73.00,...,NaN,NaN,73.00,0.0,0.0,0.0,None,3,SUR,-100.00
70822,TQBR,2018-12-29,ТНСэнЯр-п,YRSBP,31,286800.0,68.000,55.00,68.00,67.00,...,NaN,65.00,67.00,0.0,501700.0,0.0,None,3,SUR,-9.46
70823,TQBR,2018-12-29,ЗИЛ ао,ZILL,5,26388.0,852.000,850.00,856.00,856.00,...,NaN,834.00,856.00,0.0,567356.0,0.0,None,3,SUR,-0.47


Найдем, сколько дней продавались акции каждой компании:

In [3]:
secid_num_of_entries = history_data.groupby('SECID')['TRADEDATE'].count()
secid_num_of_entries

SECID
ABBN     188
ABRD     254
ACKO      69
AFKS     254
AFLT     254
        ... 
YRSBP    254
ZILL     254
ZMZN     243
ZMZNP    243
ZVEZ     254
Name: TRADEDATE, Length: 287, dtype: int64

Акции каждой компании продавались разное количество дней.
Поэтому отфильтруем только те, что продавались каждый день рассматриваемого периода: 

In [4]:
secid_num_of_entries.value_counts().to_frame().sort_values(by='TRADEDATE', ascending=False)

,count
TRADEDATE,
254,268
251,2
243,2
236,1
234,1
196,1
195,1
188,1
140,1


In [5]:
secids_to_filter = secid_num_of_entries[secid_num_of_entries == secid_num_of_entries.max()].index.tolist()

history_data_filtered = history_data[history_data.SECID.isin(secids_to_filter)]
history_data_filtered

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
1,TQBR,2018-01-03,АбрауДюрсо,ABRD,5,70500.0,140.000,139.00,143.00,139.00,...,151.50,151.50,139.00,591650.0,515000.0,591650.0,None,3,SUR,-4.79
3,TQBR,2018-01-03,Система ао,AFKS,5071,256119052.0,12.485,12.26,12.55,12.35,...,12.38,12.38,12.35,256119052.0,256119052.0,256119052.0,None,3,SUR,2.15
4,TQBR,2018-01-03,Аэрофлот,AFLT,3163,296982430.0,139.100,138.60,140.95,140.40,...,139.65,139.65,140.40,296982430.0,296982430.0,296982430.0,None,3,SUR,1.41
5,TQBR,2018-01-03,AGRO-гдр,AGRO,952,9941299.0,583.000,561.00,583.00,570.00,...,567.00,567.00,570.00,9941299.0,9941299.0,9941299.0,None,3,SUR,0.26
6,TQBR,2018-01-03,Акрон,AKRN,136,2102115.0,3900.000,3895.00,3942.00,3930.00,...,3922.00,3922.00,3930.00,2102115.0,2102115.0,2102115.0,None,3,SUR,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70820,TQBR,2018-12-29,Yandex clA,YNDX,3314,91724595.5,1920.500,1920.00,1934.00,1930.00,...,1928.00,1928.00,1930.00,91724595.5,91724595.5,91724595.5,None,3,SUR,0.73
70821,TQBR,2018-12-29,ТНСэнЯр,YRSB,0,0.0,NaN,NaN,NaN,73.00,...,NaN,NaN,73.00,0.0,0.0,0.0,None,3,SUR,-100.00
70822,TQBR,2018-12-29,ТНСэнЯр-п,YRSBP,31,286800.0,68.000,55.00,68.00,67.00,...,NaN,65.00,67.00,0.0,501700.0,0.0,None,3,SUR,-9.46
70823,TQBR,2018-12-29,ЗИЛ ао,ZILL,5,26388.0,852.000,850.00,856.00,856.00,...,NaN,834.00,856.00,0.0,567356.0,0.0,None,3,SUR,-0.47


Проверим успешность фильтрации, то есть что акции каждой компании в таблице продавались в течение одно и того же количества дней:

In [8]:
history_data_filtered.groupby('SECID')['TRADEDATE'].count().unique()

array([254])

In [14]:
async def get_moex_index_data(start_date, end_date):
    # history_data = pd.DataFrame()
    # dates = pd.date_range(start_date, end_date, freq='D')
    
    async with aiohttp.ClientSession() as session:
        
        # for date in dates:
            # date = date.strftime('%Y-%m-%d')
            
        prefix="https://iss.moex.com"
        url = f'/iss/history/engines/stock/markets/index/securities/IMOEX.json?from={start_date}&till={end_date}&sort=TRADEDATE&order=asc'
        
        # filtering columns of downloaded data
        # arguments = {"history.columns": ("BOARDID," "TRADEDATE," "SHORTNAME," "SECID," "NUMTRADES," "VALUE,"
        #                                     "OPEN," "LOW," "HIGH," "LEGALCLOSEPRICE," "WAPRICE," "CLOSE," "VOLUME")}
        
        iss = aiomoex.ISSClient(session, 
                                prefix + url,
                                # arguments
                                )
        data = await iss.get_all()
        data_df = pd.DataFrame(data['history'])
        # history_data = pd.concat([history_data, data_df], axis=0, ignore_index=True)
        
        return data_df

history_index_data = await get_moex_index_data(start_date='2018-01-01', end_date='2018-12-31')

In [15]:
history_index_data

,BOARDID,SECID,TRADEDATE,SHORTNAME,NAME,CLOSE,OPEN,HIGH,LOW,VALUE,DURATION,YIELD,DECIMALS,CAPITALIZATION,CURRENCYID,DIVISOR,TRADINGSESSION,VOLUME
0,SNDX,IMOEX,2018-01-03,Индекс МосБиржи,Индекс МосБиржи,2152.97,2111.24,2152.97,2111.24,2.107610e+10,0,0,2,9492304557100,RUB,4.408926e+09,3,None
1,SNDX,IMOEX,2018-01-04,Индекс МосБиржи,Индекс МосБиржи,2197.60,2153.87,2197.60,2153.87,3.653125e+10,0,0,2,9689074992110,RUB,4.408926e+09,3,None
2,SNDX,IMOEX,2018-01-05,Индекс МосБиржи,Индекс МосБиржи,2207.41,2196.69,2207.41,2188.35,2.598557e+10,0,0,2,9732313882890,RUB,4.408926e+09,3,None
3,SNDX,IMOEX,2018-01-09,Индекс МосБиржи,Индекс МосБиржи,2225.39,2211.40,2242.87,2211.40,4.438950e+10,0,0,2,9811559695860,RUB,4.408926e+09,3,None
4,SNDX,IMOEX,2018-01-10,Индекс МосБиржи,Индекс МосБиржи,2232.27,2227.33,2233.63,2213.77,4.008192e+10,0,0,2,9841904816640,RUB,4.408926e+09,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,SNDX,IMOEX,2018-12-25,Индекс МосБиржи,Индекс МосБиржи,2290.86,2318.43,2318.43,2258.27,1.856123e+10,0,0,2,9487797140400,RUB,4.141588e+09,3,None
250,SNDX,IMOEX,2018-12-26,Индекс МосБиржи,Индекс МосБиржи,2308.85,2291.98,2315.98,2291.98,1.494648e+10,0,0,2,9562322669710,RUB,4.141588e+09,3,None
251,SNDX,IMOEX,2018-12-27,Индекс МосБиржи,Индекс МосБиржи,2327.81,2316.66,2340.63,2316.66,2.811124e+10,0,0,2,9640819592920,RUB,4.141588e+09,3,None
252,SNDX,IMOEX,2018-12-28,Индекс МосБиржи,Индекс МосБиржи,2358.50,2329.96,2369.24,2329.96,2.367390e+10,0,0,2,9767947486870,RUB,4.141588e+09,3,None


In [27]:
history_data_filtered.to_csv('history_data.csv', encoding='utf-16', index=False)
history_index_data.to_csv('index_data.csv', encoding='utf-16', index=False)